In [8]:
import re, glob
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path

from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel
from nilearn.glm.contrasts import compute_fixed_effects
from nilearn.plotting import plot_stat_map, plot_design_matrix

# --- PATHS ---
FMRIPREP_ROOT = Path("/neurospin/motif-stroke/7T_protocol/pilots/derivatives/fmriprep")
EVENTS_DIR    = Path("/volatile/home/sb283337/Bureau/7T-fMRI-Motor-Stroke/data/events")
LOGS_DIR      = Path("/volatile/home/sb283337/Bureau/7T-fMRI-Motor-Stroke/data/log")
SEQ_DIR       = Path("/volatile/home/sb283337/Bureau/7T-fMRI-Motor-Stroke/data/seq")
RESULTS_DIR   = Path("/volatile/home/sb283337/Bureau/7T-fMRI-Motor-Stroke/results_industrial")

# --- SETTINGS ---
TASK, SPACE, TR = "motif4limbs", "MNI152NLin2009cAsym", 2.0
BLANK_S = 1.0
PAUSE_S = 4.8
STIM_S  = 2.2 
RESP_DUR_S = 0.2
SMOOTHING_FWHM = 3.5

# --- CONTRASTS
CONTRASTS = {
    "task_gt_baseline": "0.25*(main_gauche + main_droite + pied_gauche + pied_droit)",
    "hand_vs_foot": "0.5*(main_gauche + main_droite) - 0.5*(pied_gauche + pied_droit)",
    "global_right_vs_left": "0.5*(main_droite + pied_droit) - 0.5*(main_gauche + pied_gauche)",
    "right_vs_left_hand": "main_droite - main_gauche",
    "right_vs_left_foot": "pied_droit - pied_gauche"

}


def get_motif_files(sub_id):
    func_dir = FMRIPREP_ROOT / f"sub-{sub_id}" / "func"
    pattern = f"sub-{sub_id}_task-{TASK}_dir-*_run-*_space-{SPACE}_desc-preproc_bold.nii.gz"
    bolds = sorted(func_dir.glob(pattern))
    return [{
        "run": re.search(r"run-(\d+)", b.name).group(1),
        "dir": re.search(r"dir-([a-z]+)", b.name).group(1),
        "bold": b,
        "mask": func_dir / b.name.replace("desc-preproc_bold.nii.gz", "desc-brain_mask.nii.gz"),
        "conf": func_dir / b.name.replace(f"space-{SPACE}_desc-preproc_bold.nii.gz", "desc-confounds_timeseries.tsv")
    } for b in bolds]



In [9]:
# A "Menu" of coordinates for different body parts (MNI Space)
ROI_COORDS = {
    "hand_knob":    (0, -30, 60),    # Classic view for Hand/Finger areas
    "medial_wall":  (0, -30, 70),    # High axial view for Feet (closer to the midline)
    "motor_strip":  (0, -25, 55),    # Slightly lower/forward to catch the full M1/S1 range
    "lateral_view": (-40, -20, 50)   # Side view if you want to check Left M1 specifically
}

# Mapping contrasts to their best viewing coordinates
CONTRAST_VIEWS = {
    "task_gt_baseline":     "motor_strip",   # Show everything that moved
    "hand_vs_foot":         "hand_knob",     # Hand knob is the best reference here
    "global_right_vs_left": "motor_strip",   # Show the whole left vs right brain split
    "right_vs_left_hand":   "hand_knob",     # Focus on the hand area
    "right_vs_left_foot":   "medial_wall"    # Focus on the foot area (near the top/middle)
}


In [10]:
def get_motif_files(sub_id):
    """Strictly finds motif4limbs tasks and extracts run/dir."""
    func_dir = FMRIPREP_ROOT / f"sub-{sub_id}" / "func"
    # Ensure we use the exact task name
    pattern = f"sub-{sub_id}_task-{TASK}_dir-*_run-*_space-{SPACE}_desc-preproc_bold.nii.gz"
    bolds = sorted(func_dir.glob(pattern))
    
    run_data = []
    for b in bolds:
        # Improved Regex to catch 'dir' and 'run'
        run_match = re.search(r"run-(\d+)", b.name)
        dir_match = re.search(r"dir-([a-z]+)", b.name)
        
        if run_match and dir_match:
            run_num = run_match.group(1)
            direc = dir_match.group(1)
            
            run_data.append({
                "run": run_num,
                "dir": direc,
                "bold": b,
                "mask": func_dir / b.name.replace("desc-preproc_bold.nii.gz", "desc-brain_mask.nii.gz"),
                "conf": func_dir / b.name.replace(f"space-{SPACE}_desc-preproc_bold.nii.gz", "desc-confounds_timeseries.tsv")
            })
    return run_data

In [11]:
def build_events_sequence(run):
    """
    EXACT ORIGINAL LOGIC:
    - Onset depends on run ID (one file per run).
    - Uses specific BLANK_S and STIM_S increments to maintain timing.
    """
    seq_path = SEQ_DIR / f"stim_sequence_run-{int(run)}.csv"
    if not seq_path.exists():
        return None

    seq = pd.read_csv(seq_path).sort_values(["block_id", "id"])

    rows = []
    current = 0.0 # Starting t0
    current_block = None

    for _, r in seq.iterrows():
        # Block transition logic
        if current_block is None:
            current_block = r["block_id"]
        elif r["block_id"] != current_block:
            current += PAUSE_S
            current_block = r["block_id"]

        # Blank window (shifts the onset)
        current += BLANK_S

        # Record the stimulus window
        rows.append({
            "onset": current,
            "duration": STIM_S,
            "trial_type": r["block_name"],
            "modulation": 1.0
        })

        # Move to end of stimulus
        current += STIM_S

    return pd.DataFrame(rows) if rows else None

In [12]:
def build_the_design_matrix(bold_img, events, conf_path):
    """Matches your original build_design_matrix exactly."""
    n_scans = bold_img.shape[-1]
    tr = bold_img.header.get_zooms()[-1] # Uses header TR to ensure 9 drifts
    frame_times = np.arange(n_scans) * tr

    # Loading motion + outliers exactly as your original load_confounds
    conf = pd.read_csv(conf_path, sep="\t")
    motion = [c for c in ["trans_x","trans_y","trans_z","rot_x","rot_y","rot_z"] if c in conf.columns]
    outliers = [c for c in conf.columns if ("motion_outlier" in c) or ("non_steady_state" in c)]
    conf_sel = conf[motion + outliers].fillna(0.0)

    return make_first_level_design_matrix(
        frame_times=frame_times,
        events=events,
        hrf_model='glover',
        drift_model='cosine',
        high_pass=0.01,
        add_regs=conf_sel.values,
        add_reg_names=list(conf_sel.columns)
    )



In [ ]:
def fit_run_and_save_nifti(bold_path, mask_path, dm, run_dir, c_name, c_expr):
    """Only fits the GLM and saves the NIfTI files."""
    bold_img = nib.load(str(bold_path))
    tr = bold_img.header.get_zooms()[-1]
    
    model = FirstLevelModel(
        t_r=tr, mask_img=str(mask_path), hrf_model='glover', 
        drift_model='cosine', high_pass=0.01, noise_model='ar1',
        smoothing_fwhm=3.5, standardize=False, minimize_memory=False
    ).fit(bold_img, design_matrices=dm)
    
    stats = model.compute_contrast(c_expr, output_type='all')
    
    # Save the raw NIfTI files for the run
    stats['z_score'].to_filename(run_dir / f"{c_name}_zmap.nii.gz")
    stats['effect_size'].to_filename(run_dir / f"{c_name}_beta.nii.gz")
    stats['effect_variance'].to_filename(run_dir / f"{c_name}_variance.nii.gz") # Needed for fusion!
    model.r_square[0].to_filename(run_dir / "r2.nii.gz")
    
    return stats


def save_brain_viz(nifti_input, output_png, title, coords=(0, -30, 60), threshold=3.1):
    """
    Makes a PNG from a NIfTI file path OR a NIfTI object.
    """
    plot_stat_map(
        nifti_input,              # No str() here! Let Nilearn handle the object
        threshold=threshold, 
        cut_coords=coords,
        title=title, 
        colorbar=True,
        display_mode='ortho',
        black_bg=True             # Standard for 7T results
    )
    plt.savefig(output_png)
    plt.close()


def subject_fusion_and_save_data(run_stats_list, mask_path, out_dir, c_name):
    """
    Manual Fixed Effects Fusion:
    - Collects Beta and Variance from each run.
    - Computes the weighted average.
    - Saves the final NIfTI volumes.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # 1. Gather the maps from the list of results
    eff_imgs = [s['effect_size'] for s in run_stats_list]
    var_imgs = [s['effect_variance'] for s in run_stats_list]
    
    # 2. Compute the Fusion (Fixed Effects)
    # This gives us: [Effect, Variance, T-stat, Z-score]
    beta_map, var_map, t_map, z_map = compute_fixed_effects(
        eff_imgs, 
        var_imgs, 
        mask_path, 
    )
    
    # 3. Save the Total NIfTI volumes
    # Now you can save all of them to your combined_total folder
    beta_map.to_filename(out_dir / f"{c_name}_TOTAL_effect_size.nii.gz")
    var_map.to_filename(out_dir / f"{c_name}_TOTAL_variance.nii.gz")
    t_map.to_filename(out_dir / f"{c_name}_TOTAL_tstat.nii.gz")
    z_map.to_filename(out_dir / f"{c_name}_TOTAL_zscore.nii.gz")
    
    # We return the z_map object so we can plot it immediately if we want
    return z_map


### What each one tells you (for your own retrieval)

* **`beta_map`**: The "Signal." Use this to see the **percent signal change**.  "how strong was the activation?"
* **`var_map`**: The "Noise." Areas with high movement or artifacts will show up as bright spots here. It’s a great diagnostic tool to see if a run was low quality.
* **`t_map`**: The "Raw Ratio." This is simply . It’s the traditional way of looking at fMRI stats before converting to Z-scores.
* **`z_map`**: The "Final Word." This is the map you use for your **figures**. It scales the T-map so that you can use a standard threshold (like 3.1 for ).



In [ ]:
# INITIALIZE SUBJECT
SUB = "03"
METHOD = "sequence"
data_runs = get_motif_files(SUB)

# The contrast_list is derived directly from the dictionary keys
contrast_list = list(CONTRASTS.keys())

# THE MASTER LOOP
for c_name in contrast_list:
    c_expr = CONTRASTS[c_name]
    print(f"\n" + "="*50)
    print(f"STARTING ANALYSIS: {c_name}")
    print(f"FORMULA: {c_expr}")
    print("="*50)

    # Clean Folder Hierarchy
    # results/sub-03/sequence_method/hand_vs_foot/...
    SUB_METHOD_DIR = RESULTS_DIR / f"sub-{SUB}" / f"{METHOD}_method" / c_name
    SUB_METHOD_DIR.mkdir(parents=True, exist_ok=True)

    # --- STEP 2: RUN-LEVEL ANALYSIS ---
    run_stats_list = []
    
    for r in data_runs:
        # Naming the folder with the run number and the phase direction
        run_dir = SUB_METHOD_DIR / f"run-{r['run']}_dir-{r['dir']}"
        run_dir.mkdir(exist_ok=True)
        
        # Build Matrix using your original 9-drift timing logic
        ev = build_events_sequence(r['run'])
        dm = build_the_design_matrix(nib.load(str(r['bold'])), ev, r['conf'])
        
        print(f"  > Fitting Run {r['run']} ({r['dir']})...")
        # Fits GLM and saves Effect, Var, Z, and R2 Niftis
        stats = fit_run_and_save_nifti(r['bold'], r['mask'], dm, run_dir, c_name, c_expr)
        run_stats_list.append(stats)
        
        # Plot Run Viz - Coordinates are pulled automatically from your menu
        view_key = CONTRAST_VIEWS.get(c_name, "hand_knob")
        save_brain_viz(stats['z_score'], run_dir / f"{c_name}_run_viz.png", 
                       title=f"Run {r['run']}: {c_name}", coords=ROI_COORDS[view_key])

    # --- STEP 3: SUBJECT-LEVEL FUSION (The TOTAL) ---
    total_dir = SUB_METHOD_DIR / "combined_total"
    total_dir.mkdir(exist_ok=True)

    print(f"  > Fusing all runs into Total Subject Map...")
    # Retrieve Effect (Beta) and Significance (Z) from the list of results
    # We use underscores to ignore Variance and T-stat for now
    beta_total, _, _, z_total = compute_fixed_effects(
        [s['effect_size'] for s in run_stats_list],
        [s['effect_variance'] for s in run_stats_list],
        data_runs[0]['mask']
    )
    # Unpack the list manually
    beta_total = fixed_effects_results[0]
    var_total  = fixed_effects_results[1]
    t_total    = fixed_effects_results[2]
    z_total    = fixed_effects_results[3]

    # Save volumes to open in FSLeyes
    beta_total.to_filename(total_dir / f"{c_name}_TOTAL_effect.nii.gz")
    var_total.to_filename(total_dir / f"{c_name}_TOTAL_variance.nii.gz")
    t_total.to_filename(total_dir / f"{c_name}_TOTAL_tmap.nii.gz")
    z_total.to_filename(total_dir / f"{c_name}_TOTAL_zmap.nii.gz")


    # Final presentation PNG for the "Total" result
    save_brain_viz(z_total, total_dir / f"{c_name}_TOTAL_viz.png", 
                   title=f"TOTAL COMBINED: {c_name}", coords=ROI_COORDS[view_key])

print(f"\n✅ SUCCESS: All {len(contrast_list)} contrasts processed for Sub-{SUB}!")


STARTING ANALYSIS: task_gt_baseline
FORMULA: 0.25*(main_gauche + main_droite + pied_gauche + pied_droit)
[make_first_level_design_matrix] A 'modulation' column was found in the given events data and is used.
  > Fitting Run 01 (ap)...


/tmp/ipykernel_33716/2152466797.py:6: UserWarning: If design matrices are supplied, [t_r] will be ignored.
  model = FirstLevelModel(


[make_first_level_design_matrix] A 'modulation' column was found in the given events data and is used.
  > Fitting Run 02 (pa)...


/tmp/ipykernel_33716/2152466797.py:6: UserWarning: If design matrices are supplied, [t_r] will be ignored.
  model = FirstLevelModel(


  > Fusing all runs into Total Subject Map...


TypeError: compute_fixed_effects() got an unexpected keyword argument 'return_type'